# Projeto Download automático de planilhas
### Realizar o download automático das planilhas de relatório de vendas exportadas pelo sistema KP Delivery que servirão de base de dados para costrução do painel de BI

In [1]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
import os
import time

def downloadPlanilhaRelatorio():
    
    with open("config.txt", "r", encoding="utf-8") as arq:
        dados = arq.readlines()
        
    #definições
    usuario = dados[0]
    senha = dados[1]
    dataInicial = dados[2]
    dataFinal = dados[3]

    #xpaths
    xpath_login = '//*[@id="UserName"]'
    xpath_senha = '//*[@id="Password"]'
    xpath_botaoEnviar = '/html/body/div[3]/div/div/form/div/div[4]/div/button'
    xpath_dataInicial = '//*[@id="DataInicial"]'
    xpath_dataFinal = '//*[@id="DataFinal"]'
    xpath_formatoExcel = '//*[@id="Formato"]/option[2]'
    xpath_botaoEmitir = '/html/body/div[3]/form/div/div[13]/div/button'

    
    servico = Service(ChromeDriverManager().install())
    navegador = webdriver.Chrome(service=servico)

    #logar no sistema
    navegador.get("https://kpdelivery.com.br/")
    navegador.find_element("xpath",xpath_login).send_keys(usuario)
    navegador.find_element("xpath",xpath_senha).send_keys(senha)
    time.sleep(5)

    #navegador.find_element("xpath",xpath_botaoEnviar).click()

    #página de relatório de clientes
    navegador.get('https://kpdelivery.com.br/RelVendasCliente')
    time.sleep(2) 
    
    #definir datas do relatório
    navegador.find_element("xpath",xpath_dataInicial).send_keys(dataInicial)
    navegador.find_element("xpath",xpath_dataFinal).send_keys(dataFinal)
    navegador.find_element("xpath",xpath_formatoExcel).click()
    time.sleep(2)

    navegador.find_element("xpath",xpath_botaoEmitir).click()
    time.sleep(7)
    print("Download realizado com sucesso!")
    
    navegador.close()
    #retornar ano e mês do relatório
    return dataInicial[6:10], dataInicial[3:5]

    

def arquivoMaisRecente():

    caminhoDownload = r"C:\Users\PC\Downloads"
    lista_arquivos = os.listdir(caminhoDownload)
    lista_datas = []

    for arquivo in lista_arquivos:
        if ".xlsx" in arquivo:
            data = os.path.getmtime(f"{caminhoDownload}/{arquivo}")
            lista_datas.append((data, arquivo))

    lista_datas.sort(reverse=True)

    print("Arquivo mais recente na pasta: " + lista_datas[0][1])
    return lista_datas[0][1]

def moverArquivoPastaRelatorio(nomeArquivo, ano, mes):
    caminhoFinal = r"C:\Users\PC\Google Drive\PROJETOS\rapidogas_projetos\rapidogas_projetos\BI\Dashboards em PowerBI\VENDAS"
    
    caminhoDownload = r"C:\Users\PC\Downloads"
    
    try:
        os.rename(f"{caminhoDownload}/{nomeArquivo}", f"{caminhoFinal}/VENDAS-{ano}{mes}.xlsx")
        print("Arquivo salvo: \n" + f"{caminhoFinal}/VENDAS-{ano}{mes}.xlsx")
    except FileExistsError as erro:
        os.remove(f"{caminhoFinal}/VENDAS-{ano}{mes}.xlsx")
        print("Arquivo já existente será subtituído")
        moverArquivoPastaRelatorio(nomeArquivo, ano, mes)
    
    


In [5]:
ano, mes = downloadPlanilhaRelatorio()
nomeArquivo = arquivoMaisRecente()
moverArquivoPastaRelatorio(nomeArquivo, ano, mes)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 111.0.5563
[WDM] - Get LATEST chromedriver version for 111.0.5563 google-chrome
[WDM] - Driver [C:\Users\PC\.wdm\drivers\chromedriver\win32\111.0.5563.64\chromedriver.exe] found in cache


Download realizado com sucesso!
Arquivo mais recente na pasta: RelVendasCliente.xlsx
Arquivo já existente será subtituído
Arquivo salvo: 
C:\Users\PC\Google Drive\PROJETOS\rapidogas_projetos\rapidogas_projetos\BI\Dashboards em PowerBI\VENDAS/VENDAS-202303.xlsx
